## Cerrojos
1. Acceso en exclusión mutua a un dato compartido
2. Proteger operaciones de modificación de una variable compartida
3. Un único cerrojo para todos los accesos

In [2]:
import threading
import time

cerrojo=threading.Lock()

def incrementar ():
    global contador
    for i in range(1000000):
        cerrojo.acquire()
        contador+=1
        cerrojo.release()
        
def decrementar ():
    global contador
    for i in range(1000000):
        cerrojo.acquire()
        contador-=1
        cerrojo.release()
        
if __name__=="__main__":
    contador=0
    hilo1=threading.Thread(target=incrementar)
    hilo2=threading.Thread(target=decrementar)
    hilo1.start()
    hilo2.start()
    hilo1.join()
    hilo2.join()
    
    print (contador)

0


## Semáforos
1. Cuando acceso limitado a un recurso
2. Se inicializa al valor del número de hilos de acceso simultaneo
3. Un único semaforo para todos los accesos

In [3]:
import time
import logging
import logging.handlers
import threading

LOG_FILENAME = 'logging_ejemplo.out'

my_logger = logging.getLogger('MyLogger')
my_logger.setLevel(logging.DEBUG)
handler = logging.handlers.RotatingFileHandler(
    LOG_FILENAME,
    maxBytes=20,
    backupCount=5,
)
my_logger.addHandler(handler)

class GestorConexiones(object):
    def __init__(self):
        
        self.activas=[]
        self.cerrojo=threading.Lock()
        
    def activar (self, name):
        with self.cerrojo:
            self.activas.append(name)
            print("conexiones activas: "+ str(self.activas))
            
    def desactivar (self, name):
        with self.cerrojo:
            self.activas.remove(name)
            print("conexiones activas:"+ str(self.activas))
            
def conexion(semaforo,gestor):
    with semaforo:
        nombre=threading.currentThread().getName()
        gestor.activar(nombre)
        time.sleep(0.1)
        gestor.desactivar(nombre)
        


gestor=GestorConexiones()
semaforo=threading.Semaphore(2)
for i in range (4):
    t=threading.Thread(target=conexion, name="Hilo "+str(i), args=(semaforo,gestor))
    t.start()
        
   
        

conexiones activas: ['Hilo 0']
conexiones activas: ['Hilo 0', 'Hilo 1']
conexiones activas:['Hilo 0']
conexiones activas:[]
conexiones activas: ['Hilo 2']
conexiones activas: ['Hilo 2', 'Hilo 3']
conexiones activas:['Hilo 2']
conexiones activas:[]


## Productor Consumidor. Condiciones
1. Exclusión mutua
2. protege las modificaciones de datos compartidos
3. los hilos esperan en una cola, no consumen tiempo de procesador

In [3]:
def consumir(cond):
    global variable
    """Consumirá el recurso si se cumplen las condiciones de uso """
    with cond:
        cond.wait()
        print("empieza a consumir"+ str(variable))
        
def producir(cond, valor):
    global variable
    """Actualiza el recurso para que pueda ser consumido """
    with cond:
        variable=valor
        print("pone disponible el recurso")
        cond.notify()
        
        

condicion=threading.Condition()
consumidor1=threading.Thread(name="consumidor 1", target=consumir, args=(condicion,))
consumidor2=threading.Thread(name="consumidor 2", target=consumir, args=(condicion,))
productor1=threading.Thread(name="productor 1", target=producir, args=(condicion,5))
productor3=threading.Thread(name="productor 3", target=producir, args=(condicion,12))


# Aunque esté el producto disponible, el consumidor se inicia en wait, asi que hasa que el
# proximo recurso no esté disponible, no hará nada. 
consumidor1.start()
time.sleep(3)
consumidor2.start()
time.sleep(2)
productor3.start()
productor1.start()

productor3.join()
consumidor2.join()
productor1.join()
consumidor1.join()


pone disponible el recurso
empieza a consumir12
pone disponible el recurso
empieza a consumir5
